In [ ]:
{"nbformat":4,"nbformat_minor":0,"metadata":{"colab":{"provenance":[],"gpuType":"T4"},"kernelspec":{"name":"python3","display_name":"Python 3"},"language_info":{"name":"python"},"accelerator":"GPU"},"cells":[{"cell_type":"code","execution_count":1,"metadata":{"colab":{"base_uri":"https://localhost:8080/"},"id":"SFkr9yyLIsID","executionInfo":{"status":"ok","timestamp":1744433994881,"user_tz":-330,"elapsed":30,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}},"outputId":"aa583955-a876-41ce-b22a-3f1cd75cb52e"},"outputs":[{"output_type":"stream","name":"stdout","text":["Writing mm.cu\n"]}],"source":[" %%writefile mm.cu\n","\n","#include<stdio.h>\n","\n","void setData(int* A, int* B, int N) {\n","  for (int i=0;i<N;i++) {\n","    for (int j=0;j<N;j++) {\n","      A[i * N + j] = (j + i) % 100;\n","      B[i * N + j] = (j + i) % 100;\n","    }\n","  }\n","}\n","\n","void matMul(int* A, int* B, int* C, int N) {\n","  for (int i = 0;i<N;i++) {  // i represents the row index\n","    for (int j = 0;j<N;j++) { // j represents the column index\n","      int sum = 0;\n","      for (int k = 0;k<N;k++) {\n","        sum += A[(i * N) + k] * B[(N * k) + j];\n","      }\n","      C[(i * N) + j] = sum;\n","    }\n","  }\n","}\n","\n","int main() {\n","  int N = 1000;\n","  int size = N * N * sizeof(int);\n","  int *h_a = (int *)malloc(size);\n","  int *h_b = (int *)malloc(size);\n","  int *h_c = (int *)malloc(size);\n","\n","  setData(h_a,h_b,N);\n","\n","  matMul(h_a,h_b,h_c,N);\n","\n","  for (int i = 0; i<10; i++) {\n","    for (int j = 0; j<10; j++) {\n","      printf(\"%d \",h_c[i * N + j]);\n","    }\n","    printf(\"\\n\");\n","  }\n","\n","  free(h_a);\n","  free(h_b);\n","  free(h_c);\n","\n","  return 0;\n","}"]},{"cell_type":"code","source":["!nvcc -arch=sm_75 mm.cu"],"metadata":{"id":"FjE6R0nWOAR6","executionInfo":{"status":"ok","timestamp":1744434017107,"user_tz":-330,"elapsed":2483,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}}},"execution_count":2,"outputs":[]},{"cell_type":"code","source":["%time !./a.out"],"metadata":{"colab":{"base_uri":"https://localhost:8080/"},"id":"X97xC7wazCtT","executionInfo":{"status":"ok","timestamp":1744434025942,"user_tz":-330,"elapsed":4767,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}},"outputId":"20f1bd34-80dc-4e35-e8c2-53e60a7c253f"},"execution_count":3,"outputs":[{"output_type":"stream","name":"stdout","text":["3283500 3234000 3185500 3138000 3091500 3046000 3001500 2958000 2915500 2874000 \n","3234000 3283500 3234000 3185500 3138000 3091500 3046000 3001500 2958000 2915500 \n","3185500 3234000 3283500 3234000 3185500 3138000 3091500 3046000 3001500 2958000 \n","3138000 3185500 3234000 3283500 3234000 3185500 3138000 3091500 3046000 3001500 \n","3091500 3138000 3185500 3234000 3283500 3234000 3185500 3138000 3091500 3046000 \n","3046000 3091500 3138000 3185500 3234000 3283500 3234000 3185500 3138000 3091500 \n","3001500 3046000 3091500 3138000 3185500 3234000 3283500 3234000 3185500 3138000 \n","2958000 3001500 3046000 3091500 3138000 3185500 3234000 3283500 3234000 3185500 \n","2915500 2958000 3001500 3046000 3091500 3138000 3185500 3234000 3283500 3234000 \n","2874000 2915500 2958000 3001500 3046000 3091500 3138000 3185500 3234000 3283500 \n","CPU times: user 27.7 ms, sys: 5.1 ms, total: 32.8 ms\n","Wall time: 4.72 s\n"]}]},{"cell_type":"code","source":["%%writefile mm-gpu.cu\n","\n","#include <stdio.h>\n","#include <stdlib.h>\n","\n","int N = 1500;  // Matrix size\n","\n","// Kernel function: Each block computes one row of matrix C\n","__global__ void matMulKernel(int *A, int *B, int *C, int N) {\n","    int row = blockIdx.x; // Each block corresponds to one row\n","\n","    if (row < N) {\n","        for (int j = 0; j < N; j++) { // J represents the column index\n","            int sum = 0;\n","            for (int k = 0; k < N; k++) {\n","                sum += A[row * N + k] * B[k * N + j];\n","            }\n","            C[row * N + j] = sum;\n","        }\n","    }\n","}\n","\n","// Function to initialize matrices A and B\n","void setData(int *A, int *B, int N) {\n","    for (int i = 0; i < N; i++) {\n","        for (int j = 0; j < N; j++) {\n","            A[i * N + j] = (j + i) % 100;\n","            B[i * N + j] = (j + i) % 100;\n","        }\n","    }\n","}\n","\n","int main() {\n","    int size = N * N * sizeof(int);\n","\n","    // Allocate host memory\n","    int *h_A = (int *)malloc(size);\n","    int *h_B = (int *)malloc(size);\n","    int *h_C = (int *)malloc(size);\n","\n","    // Initialize input matrices\n","    setData(h_A, h_B, N);\n","\n","    // Allocate device memory\n","    int *d_A, *d_B, *d_C;\n","    cudaMalloc((void **)&d_A, size);\n","    cudaMalloc((void **)&d_B, size);\n","    cudaMalloc((void **)&d_C, size);\n","\n","    // Copy data from host to device\n","    cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);\n","    cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);\n","\n","    // Launch kernel: N blocks, 1 thread per block\n","    matMulKernel<<<N, 1>>>(d_A, d_B, d_C, N);\n","\n","    // Copy result back to host\n","    cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);\n","\n","    // Print a small part of the result matrix for verification\n","    for (int i = 0; i < 10; i++) {\n","        for (int j = 0; j < 10; j++) {\n","            printf(\"%d \", h_C[i * N + j]);\n","        }\n","        printf(\"\\n\");\n","    }\n","\n","    // Free device memory\n","    cudaFree(d_A);\n","    cudaFree(d_B);\n","    cudaFree(d_C);\n","\n","    // Free host memory\n","    free(h_A);\n","    free(h_B);\n","    free(h_C);\n","\n","    return 0;\n","}"],"metadata":{"colab":{"base_uri":"https://localhost:8080/"},"id":"COCZbpDBb6Zc","executionInfo":{"status":"ok","timestamp":1744434031032,"user_tz":-330,"elapsed":16,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}},"outputId":"4fcac731-9c88-4c59-9c65-ff90e22069a8"},"execution_count":4,"outputs":[{"output_type":"stream","name":"stdout","text":["Writing mm-gpu.cu\n"]}]},{"cell_type":"code","source":["!nvcc -arch=sm_75 mm-gpu.cu"],"metadata":{"id":"HwIdi6r4cydO","executionInfo":{"status":"ok","timestamp":1744434038688,"user_tz":-330,"elapsed":1215,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}}},"execution_count":5,"outputs":[]},{"cell_type":"code","source":["%time !./a.out"],"metadata":{"colab":{"base_uri":"https://localhost:8080/"},"id":"fMdGpioBc5f1","executionInfo":{"status":"ok","timestamp":1744434043504,"user_tz":-330,"elapsed":724,"user":{"displayName":"Dr. Anugrah Jain","userId":"09464862443318382255"}},"outputId":"4ab93634-1c4c-4652-e8b0-7c06d59f9a23"},"execution_count":6,"outputs":[{"output_type":"stream","name":"stdout","text":["4925250 4851000 4778250 4707000 4637250 4569000 4502250 4437000 4373250 4311000 \n","4851000 4925250 4851000 4778250 4707000 4637250 4569000 4502250 4437000 4373250 \n","4778250 4851000 4925250 4851000 4778250 4707000 4637250 4569000 4502250 4437000 \n","4707000 4778250 4851000 4925250 4851000 4778250 4707000 4637250 4569000 4502250 \n","4637250 4707000 4778250 4851000 4925250 4851000 4778250 4707000 4637250 4569000 \n","4569000 4637250 4707000 4778250 4851000 4925250 4851000 4778250 4707000 4637250 \n","4502250 4569000 4637250 4707000 4778250 4851000 4925250 4851000 4778250 4707000 \n","4437000 4502250 4569000 4637250 4707000 4778250 4851000 4925250 4851000 4778250 \n","4373250 4437000 4502250 4569000 4637250 4707000 4778250 4851000 4925250 4851000 \n","4311000 4373250 4437000 4502250 4569000 4637250 4707000 4778250 4851000 4925250 \n","CPU times: user 7.31 ms, sys: 678 µs, total: 7.98 ms\n","Wall time: 705 ms\n"]}]}]}